In [ ]:
import traitlets, os 
import ipywidgets.widgets as widgets 
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from uuid import uuid1

class DataCollection:    
    
    def __init__(self):
        self.image = None
        self.blocked_button = None
        self.free_button = None 
        self.blocked_count = 0
        self.free_count = 0
        self.blocked_path_dir = 'dataset/blocked'
        self.free_path_dir = 'dataset/free'

    def view_camera(self):
        robot_camera = Camera.instance(width = 224 , height = 244 )

        self.image = widgets.Image(format = 'jpeg', width = 244, height = 244)
        camera_link = traitlets.dlink((robot_camera, 'value'), (self.image, 'value'), transform=bgr8_to_jpeg)
        display(self.image)
        
    def create_dir(self):
        try:
            os.makedirs(self.blocked_path_dir)
            os.makedirs(self.free_path_dir)
        except FileExistsError:
            print("Files Already Exists\n")
        button_layout = widgets.Layout(width='128px', height='64px')
        self.free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
        self.blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
        self.free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(self.free_path_dir)))
        self.blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(self.blocked_path_dir)))

        #display(widgets.HBox([self.free_count, self.free_button]))
        #display(widgets.HBox([self.blocked_count, self.blocked_button]))
        
    def gather_data(self):
        
        def save_photo_captured(self,directory):
            image_path = os.path.join(directory, str(uuid1()) + '.jpg')
            with open(image_path, 'wb') as f:
                f.write(self.image.value)

        def save_free(self):
            save_snapshot(self.free_path_dir)
            self.free_count.value = len(os.listdir(self.free_path_dir))
    
        def save_blocked(self):
            save_snapshot(self.blocked_path_dir)
            self.blocked_count.value = len(os.listdir(self.blocked_path_dir))
        
        
        self.free_button.on_click(lambda x: save_free())
        self.blocked_button.on_click(lambda x: save_blocked())
        display(self.image)
        display(widgets.HBox([self.free_count, self.free_button]))
        display(widgets.HBox([self.blocked_count,self.blocked_button]))
        

myPhotos = DataCollection()
myPhotos.view_camera()
myPhotos.create_dir()
myPhotos.gather_data()